In [3]:
import random
import numpy as np

class VRPTWGeneticAlgorithm:
    def __init__(self, num_vehicles, locations, demands, time_windows, depot, population_size, num_generations, mutation_rate):
        self.num_vehicles = num_vehicles
        self.locations = locations
        self.demands = demands
        self.time_windows = time_windows
        self.depot = depot
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate

    def create_individual(self):
        unvisited = list(range(1, len(self.locations)))  # Adjusted range
        random.shuffle(unvisited)
        return [0] + unvisited

    def create_population(self):
        population = []
        for _ in range(self.population_size):
            individual = self.create_individual()
            population.append(individual)
        return population

    def calculate_fitness(self, individual):
        fitness = 0
        vehicle_routes = [[] for _ in range(self.num_vehicles)]
        vehicle_loads = [0] * self.num_vehicles
        vehicle_times = [0] * self.num_vehicles

        for customer in individual:
            demand = self.demands[customer]
            min_route_time = float('inf')
            selected_vehicle = -1

            for vehicle in range(self.num_vehicles):
                current_load = vehicle_loads[vehicle] + demand
                current_time = vehicle_times[vehicle]
                current_route = vehicle_routes[vehicle]

                if current_load <= self.depot[2] and current_time + self.locations[current_route[-1]][customer] + self.locations[customer][self.depot[0]] <= self.depot[1]:
                    route_time = current_time + self.locations[current_route[-1]][customer]
                    if route_time < min_route_time:
                        min_route_time = route_time
                        selected_vehicle = vehicle

            if selected_vehicle == -1:
                fitness += float('inf')
            else:
                vehicle_routes[selected_vehicle].append(customer)
                vehicle_loads[selected_vehicle] += demand
                vehicle_times[selected_vehicle] = min_route_time

        return fitness

    def selection(self, population):
        fitness_scores = [self.calculate_fitness(individual) for individual in population]
        total_fitness = sum(fitness_scores)
        probabilities = [fitness / total_fitness for fitness in fitness_scores]
        selected_indices = random.choices(range(len(population)), probabilities, k=2)
        return population[selected_indices[0]], population[selected_indices[1]]

    def crossover(self, parent1, parent2):
        crossover_point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:crossover_point] + [gene for gene in parent2 if gene not in parent1[:crossover_point]]
        child2 = parent2[:crossover_point] + [gene for gene in parent1 if gene not in parent2[:crossover_point]]
        return child1, child2

    def mutation(self, individual):
        for i in range(1, len(individual)):
            if random.random() < self.mutation_rate:
                j = random.randint(1, len(individual) - 1)
                individual[i], individual[j] = individual[j], individual[i]
        return individual

    def evolve_population(self, population):
        new_population = []
        while len(new_population) < self.population_size:
            parent1, parent2 = self.selection(population)
            child1, child2 = self.crossover(parent1, parent2)
            child1 = self.mutation(child1)
            child2 = self.mutation(child2)
            new_population.append(child1)
            new_population.append(child2)
        return new_population
    def find_best_solution(self):
        population = self.create_population()
        best_solution = None
        best_fitness = float('inf')
        for _ in range(self.num_generations):
            population = self.evolve_population(population)
            fitness_scores = [self.calculate_fitness(individual) for individual in population]
            min_fitness = min(fitness_scores)
            if min_fitness < best_fitness:
                best_fitness = min_fitness
                best_solution = population[fitness_scores.index(min_fitness)]
        return best_solution

# Main program
num_vehicles = 3
locations = [
    [0, 0],  # Depot coordinates
    [1, 2],
    [3, 4],
    [5, 6],
    [7, 8]
]

demands = [0, 2, 1, 4, 3]
time_windows = [
    [0, 10],
    [5, 15],
    [10, 20],
    [5, 25],
    [0, 30]
]
depot = [0, 0, 10]  # [depot_x, depot_y, depot_max_time]

population_size = 100
num_generations = 500
mutation_rate = 0.02

ga = VRPTWGeneticAlgorithm(num_vehicles, locations, demands, time_windows, depot, population_size, num_generations, mutation_rate)
best_solution = ga.find_best_solution()

print("Best solution:", best_solution)



IndexError: list index out of range